In [1]:
import pandas as pd
import numpy as np
import torch
import torchtext
from torchtext import data
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import torch.optim as optim
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import os
import nltk
from nltk import word_tokenize
from tqdm import tqdm
from tqdm.notebook import tqdm
from IPython.display import display
from transformers import AutoTokenizer
from gensim.models import Word2Vec
import time
# import tensorflow
# from tensorflow import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences



tqdm.pandas()

import warnings as wrn
wrn.filterwarnings('ignore')

SEED = 7
torch.manual_seed(SEED)
torch.backends.cuda.deterministic = True
nltk.download('punkt')  # Download the Punkt tokenizer


/ext3/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-11-11 16:34:38.554303: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-11 16:34:39.155800: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-11 16:34:40.126193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 16:34:40.126231: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to r

True

In [2]:
train = pd.read_csv('/vast/amr10211/sc-train.csv')
val = pd.read_csv('/vast/amr10211/sc-val.csv')
test = pd.read_csv('/vast/amr10211/sc-test.csv')

In [3]:
# def Word2Vec_embeddings(corpus, model_file_path):
#     # Tokenize the sentences
#     tokenized_corpus = [word_tokenize(doc.lower()) for doc in corpus]

#     # Train Word2Vec model
#     model = Word2Vec(sentences=tokenized_corpus, vector_size=50, window=5, sg=1, min_count=1)
    
#     #Save model
#     model.save(model_file_path)
#     return model

In [8]:
#Concat all text for Word2Vec model
# all_text = list(train['decision_text'])+list(val['decision_text'])+list(test['decision_text'])

In [10]:
#Train Word2Vec model
# w2v_model = Word2Vec_embeddings(tqdm(all_text),"w2v_model.bin")

  0%|          | 0/26425 [00:00<?, ?it/s]

In [3]:
#Load Word2Vec model
w2v_model = Word2Vec.load("w2v_model.bin")

In [14]:
# # Get the vector for a specific word
# vector = w2v_model.wv['america']
# print(f"Vector: {vector}")

# Find similar words
similar_words = w2v_model.wv.most_similar('justice', topn=5)
print(f"Words similar: {similar_words}")

Words similar: [('brennan-', 0.8825850486755371), ('marshall', 0.8814927339553833), ('blacicmun', 0.8791638612747192), ('mr.', 0.8783459067344666), ('stewart', 0.87803715467453)]


In [13]:
MAX_SEQUENCE_LENGTH = 10000

In [5]:
def word2token(word):
    try:
        return w2v_model.wv.key_to_index[word]
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0

In [14]:
def tokenize_vectorize(data, model, MAX_SEQUENCE_LENGTH):
    start = time.time()
    # Tokenize the documents
    tokenized_data =[word_tokenize(doc.lower()) for doc in data]
    token = time.time()
    print('tokenize: ', token-start)
    # Assuming 'model' is your Word2Vec model
    vector_size = model.vector_size  # Get the size of the word vectors

    # Convert tokens to word vectors
    # word_vectors = [[model.wv[token] for token in doc if token in model.wv] for doc in tokenized_data]
    word_vectors = [[word2token(word) for word in doc] for doc in tokenized_data]

    vector = time.time()
    print('vectorize: ', vector-token)
    
    padded_word_vectors = pad_sequences(word_vectors, maxlen=MAX_SEQUENCE_LENGTH, padding='post', value=0.0, dtype = float)

    return padded_word_vectors

In [ ]:
X_train = tokenize_vectorize(tqdm(train['decision_text']), w2v_model, MAX_SEQUENCE_LENGTH)
X_val = tokenize_vectorize(tqdm(val['decision_text']), w2v_model, MAX_SEQUENCE_LENGTH)
# X_test = tokenize_vectorize(tqdm(test['decision_text']), w2v_model, MAX_SEQUENCE_LENGTH)

y_train = np.array(train['scdb_decision_direction'])
y_val = np.array(val['scdb_decision_direction'])
y_train = (y_train == 1.0).astype(int)
y_val = (y_val == 1.0).astype(int)

# # # y_test = test['scdb_decision_direction']
                 
# # train_df = pd.DataFrame([X_train,y_train], columns = ['text','label'])
# val_df = pd.DataFrame(pd.concat([pd.Series(X_val), pd.Series(y_val)],axis=1))
# val_df.rename(columns = {0:'text',1:'label'}, inplace = True)
# # test_df = pd.DataFrame([X_test,y_test], columns = ['text','label'])


  0%|          | 0/21107 [00:00<?, ?it/s]

In [9]:
print(len(y_train))


1000


In [53]:
# # Create a custom dataset for the Pandas DataFrame
# class CustomDataset(Dataset):
#     def __init__(self, dataframe):
#         self.data = dataframe

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, index):
#         text = self.data.iloc[index]['text']
#         label = self.data.iloc[index]['label']
#         # You can implement any additional data processing here
#         return {'text': text, 'label': label}

In [91]:
# val_dataset = CustomDataset(val_df)

In [93]:
# def pad_collate(batch):
#     (xx, yy) = zip(*batch)
#     x_lens = [len(x) for x in xx]
#     y_lens = [len(y) for y in yy]

#     xx_pad = pad_sequence(xx, batch_first=True, padding_value=0)
#     yy_pad = pad_sequence(yy, batch_first=True, padding_value=0)

#     return xx_pad, yy_pad, x_lens, y_lens

In [94]:
# val_dataloader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=True, collate_fn=pad_collate)


In [8]:
w2v_weights = w2v_model.wv.vectors
vocab_size, embedding_size = w2v_weights.shape

print("Vocabulary Size: {} - Embedding Dim: {}".format(vocab_size, embedding_size))

Vocabulary Size: 625545 - Embedding Dim: 50


In [16]:
model = Sequential()

# Keras Embedding layer with Word2Vec weights initialization
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_size,
                    weights=[w2v_weights],
                    input_length=MAX_SEQUENCE_LENGTH,
                    mask_zero=True,
                    trainable=False))

model.add(Bidirectional(LSTM(10)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=3, batch_size=64,
                    validation_data=(X_val, y_val), verbose=1)

Epoch 1/3
79/79 [==============================] - 608s 8s/step - loss: 0.6731 - accuracy: 0.5726 - val_loss: 0.6481 - val_accuracy: 0.6180
Epoch 2/3
10/79 [==>...........................] - ETA: 8:40 - loss: 0.6365 - accuracy: 0.6094

KeyboardInterrupt: 

In [107]:
# def build_vocabulary(X):
#     print("Building Vocabulary ...")
#     vocab = build_vocab_from_iterator(
#         X,
#         min_freq=1,
#         specials=["<s>", "</s>", "<blank>", "<unk>"],
#     )

#     # Default index to return when OOV (out of vocab)
#     vocab.set_default_index(vocab["<unk>"])

#     return vocab

In [108]:
# def load_vocab(X):
#     if not os.path.exists("vocab.pt"):
#         vocab = build_vocabulary(X)
#         torch.save((vocab), "vocab.pt")
#     else:
#         vocab = torch.load("vocab.pt")
#     print("Finished.\nVocabulary sizes:")
#     print(len(vocab))
#     return vocab

In [109]:
# vocab = load_vocab(X)


Finished.
Vocabulary sizes:
563250


In [98]:
max_length = len(max(X, key=len))
print('Max Sequence Length:',max_length)

NameError: name 'X' is not defined

In [99]:
class LSTMBinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMBinaryClassifier, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Define the output layer with sigmoid activation for binary classification
        self.fc = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self,forx):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, _ = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)

        return out.squeeze(1)  # Squeeze the output to make it compatible with binary cross-entropy loss

# Example usage
input_size = 100000  # Assuming each word is represented as a 300-dimensional vector
hidden_size = 64  # Number of hidden units in the LSTM
num_layers = 2  # Number of LSTM layers

# Create an instance of the LSTM binary classifier model
model = LSTMBinaryClassifier(input_size, hidden_size, num_layers)

# Print the model architecture
print(model)

LSTMBinaryClassifier(
  (lstm): LSTM(100000, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [100]:
# model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-4)
criterion = nn.BCELoss()

In [103]:
def train_model(model,iterator,optimizer,criterion):
    
    epoch_loss = 0.0
    epoch_acc = 0.0
    
    model.train()
    
    for batch in iterator:
        
        # cleaning the cache of optimizer
        optimizer.zero_grad()
        
        text,text_lengths = batch.text
        
        # forward propagation and squeezing
        predictions = model(text,text_lengths).squeeze()
        
        # computing loss / backward propagation
        loss = criterion(predictions,batch.type)
        loss.backward()
        
        # accuracy
        acc = binary_accuracy(predictions,batch.type)
        
        # updating params
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    # It'll return the means of loss and accuracy
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [105]:
train_model(model,torch.tensor(val_dataloader),optimizer,criterion)

RuntimeError: Could not infer dtype of DataLoader